In [1]:
import numpy as np
from plotly import offline
from plotly import graph_objs as go
from matplotlib import pyplot
import torch
from PIL import Image
import plotly.express as px
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
color = px.colors.sequential.Rainbow[::-1]
marker_size = 3

def points_to_image(exp_name,threshold):
    dist = np.loadtxt("3D_Experements/"+exp_name+"_x.txt")
    upper_angle = np.loadtxt("3D_Experements/"+exp_name+"_y.txt")
    lower_angle = np.loadtxt("3D_Experements/"+exp_name+"_z.txt")

    print(np.amax(upper_angle*180/np.pi))
    print(np.amin(upper_angle*180/np.pi))
    print(np.amax(lower_angle*180/np.pi))
    print(np.amin(lower_angle*180/np.pi))
    

    #x , y , z = np.array(dist)*np.cos(uAngel)*np.sin(lAngel) , np.array(dist)*np.cos(uAngel)*np.cos(lAngel) , np.array(dist)*np.sin(uAngel)

    my_sample_x = np.array(dist)*np.cos(upper_angle)*np.sin(lower_angle)
    my_sample_y =  np.array(dist)*np.cos(upper_angle)*np.cos(lower_angle)
    my_sample_z =  np.array(dist)*np.sin(upper_angle)
    
    my_sample_x = my_sample_x[~np.isnan(my_sample_x)]
    my_sample_y = my_sample_y[~np.isnan(my_sample_y)]
    my_sample_z = my_sample_z[~np.isnan(my_sample_z)]

    
    min_depth = np.amin(my_sample_y)
    max_depth = (min_depth + threshold)
    indx = my_sample_y <= (min_depth + threshold)

    my_sample_x = my_sample_x[indx]
    my_sample_y = my_sample_y[indx]
    my_sample_z = my_sample_z[indx]
    
   
    
    
    
    
    
    my_sample_y = 1-  my_sample_y / np.max(my_sample_y)
    

    x_min = np.min(my_sample_x)
    x_max = np.max(my_sample_x)
    y_min = np.min(my_sample_y)
    y_max = np.max(my_sample_y)
    z_min = np.min(my_sample_z)
    z_max = np.max(my_sample_z)
    width = 1+(x_max - x_min).astype(int)
    hight = 1+(z_max - z_min).astype(int)

    # my_sample_y = my_sample_y + (1-y_max)
    print("img :: ",width,hight)
    img = np.zeros((width,hight))
    
    for i in range(len(my_sample_x)):
        x = (my_sample_x[i] - x_min).astype(int)
        z = (my_sample_z[i] - z_min).astype(int)
#         print("x,z :: ",x,z)
        img[x,z] = my_sample_y[i]
        
#     img = Image.fromarray(img)
    print(len(my_sample_x))
    
    kernel = np.ones((5,5), np.uint8)
 
    # The first parameter is the original image,
    # kernel is the matrix with which image is
    # convolved and third parameter is the number
    # of iterations, which will determine how much
    # you want to erode/dilate a given image.
    img_erosion = cv2.erode(img, kernel, iterations=1)
    img_dilation = cv2.dilate(img, kernel, iterations=5)

    cv2.imshow('Input', img)
    cv2.imshow('Erosion', img_erosion)
    cv2.imshow('Dilation', img_dilation)

    cv2.waitKey(0)
#     img.show()
    
def plot_3D (exp_name , depth_axis):
    dist = np.loadtxt("3D_Experements/"+exp_name+"_x.txt")
    upper_angle = np.loadtxt("3D_Experements/"+exp_name+"_y.txt")
    lower_angle = np.loadtxt("3D_Experements/"+exp_name+"_z.txt")

    print(np.amax(upper_angle*180/np.pi))
    print(np.amin(upper_angle*180/np.pi))
    print(np.amax(lower_angle*180/np.pi))
    print(np.amin(lower_angle*180/np.pi))
    

    #x , y , z = np.array(dist)*np.cos(uAngel)*np.sin(lAngel) , np.array(dist)*np.cos(uAngel)*np.cos(lAngel) , np.array(dist)*np.sin(uAngel)

    my_sample_x = np.array(dist)*np.cos(upper_angle)*np.sin(lower_angle)
    my_sample_y =  np.array(dist)*np.cos(upper_angle)*np.cos(lower_angle)
    my_sample_z =  np.array(dist)*np.sin(upper_angle)
    
    my_sample_x = my_sample_x[~np.isnan(my_sample_x)]
    my_sample_y = my_sample_y[~np.isnan(my_sample_y)]
    my_sample_z = my_sample_z[~np.isnan(my_sample_z)]

    df = pd.DataFrame(my_sample_x,columns=['X (mm)'])
    df['Y (mm)'] = my_sample_y
    df['Z (mm)'] = my_sample_z
    df.head()
    min_depth = 0
    max_depth = 0
    if depth_axis == 'x':
        df['Depth'] = my_sample_x
        max_depth = np.amax(my_sample_x)
        min_depth = np.amin(my_sample_x)

    elif depth_axis == 'y':
        df['Depth'] = my_sample_y
        max_depth = np.amax(my_sample_y)
        min_depth = np.amin(my_sample_y)

    elif depth_axis == 'z':
        df['Depth'] = my_sample_z
        max_depth = np.amax(my_sample_z)
        min_depth = np.amin(my_sample_z)
        

    else:
        print("invalid depth axis")
        return
   

    fig = px.scatter_3d(df, x='X (mm)', y='Y (mm)', z='Z (mm)', color='Depth', title="ٌRadar Point Cloud" , range_color=[min_depth-200,max_depth+200],color_continuous_scale=color , opacity=1)
    fig.update_traces(marker=dict(size=marker_size, line=dict(width=0))) 
    fig.show()
    
    
def plot_3D_threshold (exp_name ,folder_indx , depth_axis ,threshold):
    folder = ["3D_Experements", "flat_Experements"]
    dist = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_x.txt")
    upper_angle = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_y.txt")
    lower_angle = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_z.txt")

    print(np.amax(upper_angle*180/np.pi))
    print(np.amin(upper_angle*180/np.pi))
    print(np.amax(lower_angle*180/np.pi))
    print(np.amin(lower_angle*180/np.pi))
    

    #x , y , z = np.array(dist)*np.cos(uAngel)*np.sin(lAngel) , np.array(dist)*np.cos(uAngel)*np.cos(lAngel) , np.array(dist)*np.sin(uAngel)

    my_sample_x = np.array(dist)*np.cos(upper_angle)*np.sin(lower_angle)
    my_sample_y =  np.array(dist)*np.cos(upper_angle)*np.cos(lower_angle)
    my_sample_z =  np.array(dist)*np.sin(upper_angle)
    my_sample_x = my_sample_x[~np.isnan(my_sample_x)]
    my_sample_y = my_sample_y[~np.isnan(my_sample_y)]
    my_sample_z = my_sample_z[~np.isnan(my_sample_z)]

    df = pd.DataFrame()
    
    indx = []
    min_depth = 0
    max_depth = 0
    if depth_axis == 'x':
        min_depth = np.amin(my_sample_x)
        max_depth = threshold#(min_depth + threshold)
        indx = my_sample_x<= threshold#(min_depth + threshold)
        df['Depth'] = my_sample_x[indx]

    elif depth_axis == 'y':
        min_depth = np.amin(my_sample_y)
        max_depth = threshold#(min_depth + threshold)
        indx = my_sample_y <= threshold#(min_depth + threshold)
#         print(my_sample_y , (min_depth + threshold))
#         print(type(my_sample_y),type(my_sample_y[indx]))
        df['Depth'] = my_sample_y[indx]

    elif depth_axis == 'z':
        min_depth = np.amin(my_sample_z)
        max_depth = threshold#(min_depth + threshold)
        indx = my_sample_z<= threshold#(min_depth + threshold)
        df['Depth'] = my_sample_z[indx]
        

    else:
        print("invalid depth axis")
        return
    df['X (mm)'] = my_sample_x[indx]
    df['Y (mm)'] = my_sample_y[indx]
    df['Z (mm)'] = my_sample_z[indx]
    df.head()
   

    # fig = px.scatter_3d(df, x='X (mm)', y='Y (mm)', z='Z (mm)', color='Depth', title="ٌRadar Point Cloud" , range_color=[min_depth,max_depth], range_y=[300,700],color_continuous_scale=color , opacity=1)
    fig = px.scatter_3d(df, x='X (mm)', y='Y (mm)', z='Z (mm)', color='Depth', title="ٌRadar Point Cloud" , range_color=[min_depth,max_depth],color_continuous_scale=color , opacity=1)
    fig.update_traces(marker=dict(size=marker_size, line=dict(width=0))) 
    fig.show()


def plot_2D (exp_name):
    dist = np.loadtxt("3D_Experements/"+exp_name+"_x.txt")
    upper_angle = np.loadtxt("3D_Experements/"+exp_name+"_y.txt")
    lower_angle = np.loadtxt("3D_Experements/"+exp_name+"_z.txt")

    print(np.amax(upper_angle*180/np.pi))
    print(np.amin(upper_angle*180/np.pi))
    print(np.amax(lower_angle*180/np.pi))
    print(np.amin(lower_angle*180/np.pi))
    

    #x , y , z = np.array(dist)*np.cos(uAngel)*np.sin(lAngel) , np.array(dist)*np.cos(uAngel)*np.cos(lAngel) , np.array(dist)*np.sin(uAngel)

    my_sample_x = np.array(dist)*np.cos(upper_angle)*np.sin(lower_angle)
    my_sample_y =  np.array(dist)*np.cos(upper_angle)*np.cos(lower_angle)
    my_sample_z =  np.array(dist)*np.sin(upper_angle)

    minx = np.min(my_sample_x)
    miny = np.min(my_sample_y)
    minz = np.min(my_sample_z)

    maxx = np.max(my_sample_x)
    maxy = np.max(my_sample_y)
    maxz = np.max(my_sample_z)

    # img =np.zeros(int(maxx-minx),int(maxz-minz))
    # img =np.zeros((int(maxx-minx),int(maxz-minz)), dtype=np.float64)
    # img =np.zeros(len(my_sample_x),len(my_sample_z)), dtype=np.float64)

    # for i in range(len(my_sample_x) - 1):
    #     img[int(my_sample_x[i] - minx -1)][int(my_sample_z[i] - minz -1)] = (my_sample_y[i] / maxy)*255
    
    trace = go.Surface(x = my_sample_x, y = my_sample_y, z =my_sample_z )
    data = [trace]
    layout = go.Layout(title = '3D Surface plot')
    fig = go.Figure(data = data)
    plot(fig)
    offline.init_notebook_mode(connected=False)

    df3 = {'x':[1, 2, 3, 4, 5],'y':[10, 20, 30, 40, 50],'z': [[5, 4, 3, 2, 1]]*5}
    offline.iplot(dict(data=[go.Surface(x=df3['x'], y=df3['y'], z=df3['z'])]))

def plot_var(threshold):
    exp_p = ["gedo_14_7",
             "waleed_14_7",
             "ta7a_14_7",
             "nassar_14_7"]

    exp_n = ["metal_dish",
            "painting"]

    var_all = []
    count = 0
    color = []

    for i in range(len(exp_p)):
        for j in range(1000):
            exp_name = exp_p[i] + "_" + str(j)
            dist = np.loadtxt("3D_Experements/"+exp_name+"_x.txt")
            upper_angle = np.loadtxt("3D_Experements/"+exp_name+"_y.txt")
            lower_angle = np.loadtxt("3D_Experements/"+exp_name+"_z.txt")
            my_sample_y =  np.array(dist)*np.cos(upper_angle)*np.cos(lower_angle)

            min_depth = np.amin(my_sample_y)
            max_depth = (min_depth + threshold)
            indx = my_sample_y <= (min_depth + threshold)
            my_sample_y = my_sample_y[indx]
            var_all.append(np.var(my_sample_y))
            count+=1
            color.append(0)
            # print(count)

    
    for i in range(len(exp_n)):
        for j in range(1000):
            exp_name = exp_n[i] + "_" + str(j)
            dist = np.loadtxt("flat_Experements/"+exp_name+"_x.txt")
            upper_angle = np.loadtxt("flat_Experements/"+exp_name+"_y.txt")
            lower_angle = np.loadtxt("flat_Experements/"+exp_name+"_z.txt")
            my_sample_y =  np.array(dist)*np.cos(upper_angle)*np.cos(lower_angle)
            # var_all.append(np.var(my_sample_y))

            min_depth = np.amin(my_sample_y)
            max_depth = (min_depth + threshold)
            indx = my_sample_y <= (min_depth + threshold)
            my_sample_y = my_sample_y[indx]
            var_all.append(np.var(my_sample_y))
            color.append(1)
            count+=1
            # print(count)

    x = [num for num in range(0, len(var_all), 1)]
    # a = np.array([x,var_all])

    categories = np.array(color)

    colormap = np.array(['g', 'r'])

    plt.scatter(np.array(x), np.array(var_all), s=100, c=colormap[categories])

    # plt.savefig('ScatterClassPlot.png')
    # plt.show()
     
    
    # plt.plot(x, var_all,'o',color = color)
    plt.xlabel('X')
    plt.ylabel('var')
    plt.grid(True)

    plt.plot()
from skimage.filters import threshold_minimum

min_dist = 0
max_dist=800
bin_size = 4
def plot_hist(exp_name,folder_indx):
    np.seterr(divide='ignore', invalid='ignore')
    folder = ["3D_Experements", "flat_Experements"]
    dist = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_x.txt")
    upper_angle = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_y.txt")
    lower_angle = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_z.txt")

    my_sample_y = np.array(dist)*np.cos(upper_angle)*np.cos(lower_angle)
    # plt.hist(my_sample_y , bins = [i for i in range(200,800,10)]) 
    # plt.title("histogram") 
    # thresh_min = threshold_minimum(my_sample_y)
    # print(thresh_min)
    # plt.axvline(thresh_min, color='r')
    # plt.show()  

    #######################################
    


    # Set total number of bins in the histogram
    bins_num = [i for i in range(min_dist,max_dist,bin_size)]#256

    
    # Get the image histogram
    n = np.histogram(my_sample_y , bins = bins_num) 
    hist = n[0]
    bin_edges = n[1]
#     print (len(hist))
#     print (len(bin_edges))
    
    # Get normalized histogram if it is required
#     if is_normalized:
#     hist = np.divide(hist.ravel(), hist.max())
#     hist = preprocessing.normalize([hist])
    # Calculate centers of bins
    bin_mids = (bin_edges[:-1] + bin_edges[1:]) / 2.
    
    # print ("bin_edges[:-1]" , bin_edges[:-1])
    # print ("bin_edges[1:]", bin_edges[1:])
    # Iterate over all thresholds (indices) and get the probabilities w1(t), w2(t)
    weight1 = np.cumsum(hist)
    
    weight2 = np.cumsum(hist[::-1])[::-1]
    
    # Get the class means mu0(t)
    mean1 = np.cumsum(hist * bin_mids) / weight1
    # Get the class means mu1(t)
    mean2 = (np.cumsum((hist * bin_mids)[::-1]) / weight2[::-1])[::-1]

    mean1 = np.nan_to_num(mean1)
    mean2 = np.nan_to_num(mean2)
    

    
    inter_class_variance = weight1[:-1] * weight2[1:] * (mean1[:-1] - mean2[1:]) ** 2

    # Maximize the inter_class_variance function val
    index_of_max_val = np.argmax(inter_class_variance)

    threshold = bin_mids[:-1][index_of_max_val]
#     print("Otsu's algorithm implementation thresholding result: ", threshold)

    
#     plt.title("histogram") 
#     plt.axvline(threshold, color='r')
#     plt.show()

    hist = [ hist[i] if bin_edges[i] < threshold else 0 for i in range(hist.size)]
    my_sample_y = my_sample_y[my_sample_y < threshold]
    
    total_value = np.sum(hist)
    hist_prob = hist / total_value
    hist_cumsum = np.cumsum(hist_prob)
    new_hist = hist_cumsum * 800
#     print ("hist_cumsum:: ",hist_cumsum)
    new_hist = np.floor(new_hist).astype(int)
#     print ("new_hist:: ",new_hist)
    new_my_sample_y = np.zeros(len(my_sample_y))
    for idx in range (len(my_sample_y)):
        for i in range(len(bin_edges)):
            if bin_edges[i]> my_sample_y[idx] :
#                 print("my_sample_y",my_sample_y[idx],"bin_edges",bin_edges[i],"i",i,"new_hist" ,new_hist[i-1])
                new_my_sample_y[idx] = new_hist[i-1]
                break
#     print("len_ny",new_my_sample_y.shape)
#     print("len_oy",my_sample_y.shape)
    
#     plt.title("histogram_2")
    
    n = np.histogram(new_my_sample_y , bins = bins_num) 
    hist_new = n[0]
    bin_edges_new = n[1]
#     plt.show()
#     x = my_sample_y
#     y = new_my_sample_y

#     bins = bins_num

#     pyplot.hist(x, bins_num, alpha=0.5, label='x')
#     pyplot.hist(y, bins_num, alpha=0.5, label='y')
#     pyplot.legend(loc='upper right')
#     pyplot.show()
    
    
#     print("oh",hist)
#     print("nh",hist_n)
#     print("oy",bin_edges)
#     print("ny",new_my_sample_y)
    return threshold,hist,bin_edges,hist_new,bin_edges_new
    


In [ ]:
def Histogram_Equalization (hist, mapping_value):
    print (hist.shape)
    print ("nassar:",hist)
    total_value = np.sum(hist)
    hist_prob = hist / total_value
    hist_cumsum = np.cumsum(hist_prob)
    new_hist = hist_cumsum * mapping_value
    print ("hist_cumsum:: ",hist_cumsum)
    new_hist = np.floor(new_hist).astype(int)
    print ("new_hist:: ",new_hist)
    new_array = np.zeros(mapping_value)
    for idx in range (len(hist)):
        new_array[new_hist[idx]+200] = hist[idx]
#         print (new_array[new_hist[idx]])
#     plt.hist(hist, bins =[i for i in range(200,800,4)] )
    print(new_array)
    plt.hist(new_my_sample_y, bins =[i for i in range(200,800,4)] )
    plt.show()


In [13]:
# plot_3D("waleed2_12_7",'y')\
# plot_3D_threshold("ta7a_14_7_20",0,'y',100000)
plot_3D_threshold("test_gedo_18_7",0,'y',600)
# points_to_image("ta7a_14_7_20",90)
# plot_hist("nassar_14_7_20",0)
# plot_hist("painting_20",1)

# plot_3D_threshold("ta7a_14_7_200",0,'y',plot_hist("ta7a_14_7_200",0)[0])
# plot_3D_threshold("painting_200",1,'y',plot_hist("painting_200",1)[0])
t , h, e = plot_hist("ta7a_14_7_100",0)
print (h)
# Histogram_Equalization (h, len(h))
# print ("old_h  ::  ",h)
# h = h[h != 0]

# print ("new_h  ::  ",h)
# print(e)
# print(t)
# # print(len(h[1][indx[:len(indx)]]))
# his = [ h[i] if e[i] < t else 0 for i in range(h.size)]
# print(his)
# print(len(h[2]))
# h = [ h[i] if i <= t else 0 for i in range(len(h))]


13.5
-13.5
13.5
-13.5


ValueError: too many values to unpack (expected 3)

In [ ]:
feature_hist_ = []
target_list_ = []
feature_hist_.append(his)
target_list_.append(1)

In [ ]:
plot_3D_threshold("test_waleed2_17_7",0,'y',520)
plot_hist("test_waleed2_17_7",0)

In [ ]:
target_list = np.loadtxt("features/temp_target_list.txt")
data_list = np.loadtxt("features/temp_data.txt")

In [100]:
# This function is used to create separated dataset files 
# for each experement 
def create_dataset():

    exp_p = [
#         "waleed_60CM", 
#             "nassar_50cm",
#             "gedo_50cm",
#              "gedo_14_7",
#                 "waleed_14_7",
#                 "ta7a_14_7",
#                 "nassar_14_7"
#         "nassar_mask_glasses"
        "gedo_mask_glass"
    ]

    exp_n = [
#         "painting_50cm_rotation2",
#         "painting_50cm_rotation",
#         "painting_50cm",
#         "painting_40cm_with_angle",
#         "painting_40cm",
#     "metal_dish",
#             "painting", 
#             "ahmed_image_40cm", 
#             "ahmed_image_40cm_with_chair_behind", 
#             "ahmed_image_50cm", 
#             "ahmed_image_50cm_with_chair_behind", 
#             "ahmed_image_60cm", 
#             "ahmed_image_60cm_with_chair_behind", 
#             "Ball_40cm", 
#             "Ball_50cm", 
#             "Ball_60cm", 
#             "basket_40cm", 
#             "basket_50cm", 
#             "basket_60cm", 
#             "vas"
#         "air"
            ]
    count = 0

#     for i in range(len(exp_n)):
#         feature_hist = []
#         target_list = []
#         for j in range(1000):
#             exp_name = exp_n[i] + "_" + str(j)
#             t , h ,e , h_n, e_n= plot_hist(exp_name,1)
#             # h = h/np.sum(h)
# #             f = [ h[n] if e[n] < t else 0 for n in range(h.size)]
#             feature_hist.append(h_n)
#             target_list.append(-1)
#             count += 1
#             print("count :: ",count)
#         feature_hist = np.array(feature_hist)
#         target_list = np.array(target_list)
        
#         file_data = open("equalized_feature/"+exp_n[i]+"_features.txt", "a")
#         np.savetxt(file_data, feature_hist)
#         file_data.close()

#         file_target_list = open("equalized_feature/"+exp_n[i]+"_target_list.txt", "a")
#         np.savetxt(file_target_list, target_list)
#         file_target_list.close()
        
    for i in range(len(exp_p)):
        feature_hist = []
        target_list = []
        for j in range(1000):
            exp_name = exp_p[i] + "_" + str(j)
            t , h ,e ,h_n , e_n = plot_hist(exp_name,0)
            # h = h/np.sum(h)
#             f = [ h[n] if e[n] < t else 0 for n in range(h.size)]
            feature_hist.append(h_n)
            target_list.append(1)
            count += 1
            print("count :: ",count)
        feature_hist = np.array(feature_hist)
        target_list = np.array(target_list)
        
        file_data = open("equalized_feature/"+exp_p[i]+"_features.txt", "a")
        np.savetxt(file_data, feature_hist)
        file_data.close()

        file_target_list = open("equalized_feature/"+exp_p[i]+"_target_list.txt", "a")
        np.savetxt(file_target_list, target_list)
        file_target_list.close()
    return "done"




In [101]:
print(create_dataset())

count ::  1
count ::  2
count ::  3
count ::  4
count ::  5
count ::  6
count ::  7
count ::  8
count ::  9
count ::  10
count ::  11
count ::  12
count ::  13
count ::  14
count ::  15
count ::  16
count ::  17
count ::  18
count ::  19
count ::  20
count ::  21
count ::  22
count ::  23
count ::  24
count ::  25
count ::  26
count ::  27
count ::  28
count ::  29
count ::  30
count ::  31
count ::  32
count ::  33
count ::  34
count ::  35
count ::  36
count ::  37
count ::  38
count ::  39
count ::  40
count ::  41
count ::  42
count ::  43
count ::  44
count ::  45
count ::  46
count ::  47
count ::  48
count ::  49
count ::  50
count ::  51
count ::  52
count ::  53
count ::  54
count ::  55
count ::  56
count ::  57
count ::  58
count ::  59
count ::  60
count ::  61
count ::  62
count ::  63
count ::  64
count ::  65
count ::  66
count ::  67
count ::  68
count ::  69
count ::  70
count ::  71
count ::  72
count ::  73
count ::  74
count ::  75
count ::  76
count ::  77
count ::

count ::  595
count ::  596
count ::  597
count ::  598
count ::  599
count ::  600
count ::  601
count ::  602
count ::  603
count ::  604
count ::  605
count ::  606
count ::  607
count ::  608
count ::  609
count ::  610
count ::  611
count ::  612
count ::  613
count ::  614
count ::  615
count ::  616
count ::  617
count ::  618
count ::  619
count ::  620
count ::  621
count ::  622
count ::  623
count ::  624
count ::  625
count ::  626
count ::  627
count ::  628
count ::  629
count ::  630
count ::  631
count ::  632
count ::  633
count ::  634
count ::  635
count ::  636
count ::  637
count ::  638
count ::  639
count ::  640
count ::  641
count ::  642
count ::  643
count ::  644
count ::  645
count ::  646
count ::  647
count ::  648
count ::  649
count ::  650
count ::  651
count ::  652
count ::  653
count ::  654
count ::  655
count ::  656
count ::  657
count ::  658
count ::  659
count ::  660
count ::  661
count ::  662
count ::  663
count ::  664
count ::  665
count 

In [ ]:
target_list = np.loadtxt("features/basket_50cm_target_list.txt")
data_list = np.loadtxt("features/basket_50cm_features.txt")
print(data_list.shape)
print(target_list.shape)

In [102]:
# In this cell we will read data in one array to train 
import numpy as np

def pre_processing(exp_name):
    target_list = np.loadtxt("equalized_feature/"+exp_name+"_target_list.txt")
    data_list = np.loadtxt("equalized_feature/"+exp_name+"_features.txt")
    for j in range(1000):
        data_list[j] = (data_list[j] - np.min(data_list[j])) / (np.max(data_list[j]) - np.min(data_list[j]))
#         temp = data_list[j]
#         temp = temp[temp != 0]
#         temp = np.trim_zeros(temp)
#         N = 59 - len(temp)
#         temp = np.pad(temp, (0, N), 'constant')
#         data_list[j] = temp
    
    print(np.sum(data_list))
    return data_list,target_list
    
    

exp_p = [
        "waleed_60CM", 
        "nassar_50cm",
        "gedo_50cm",
        "gedo_14_7",
        "waleed_14_7",
        "ta7a_14_7",
        "nassar_14_7",
        "waleed_mask_glass",
        "nassar_mask_glasses",
        "gedo_mask_glass"
        ]

exp_n = ["metal_dish",
        "painting", 
        "ahmed_image_40cm", 
        "ahmed_image_40cm_with_chair_behind", 
        "ahmed_image_50cm", 
        "ahmed_image_50cm_with_chair_behind", 
        "ahmed_image_60cm", 
        "ahmed_image_60cm_with_chair_behind",
         "air",
         "painting_50cm_rotation2",
        "painting_50cm_rotation",
        "painting_50cm",
        "painting_40cm_with_angle",
        "painting_40cm",
#          "Ball_40cm", 
#         "Ball_50cm", 
#         "Ball_60cm", 
#         "basket_40cm",
#         "basket_50cm", 
#         "basket_60cm"
#          , "vas"
        ]
         
total_data = []
total_target_lists = []
for i in range(len(exp_n)):
     
    data_list , target_list = pre_processing(exp_n[i])   
    target_list = target_list.tolist()
    data_list = data_list.tolist()
    total_data.extend(data_list)
    total_target_lists.extend(target_list)
    
for i in range(len(exp_p)):
    data_list , target_list = pre_processing(exp_p[i])   
    target_list = target_list.tolist()
    data_list = data_list.tolist()
    total_data.extend(data_list)
    total_target_lists.extend(target_list)
    
total_data = np.array(total_data)
total_target_lists = np.array(total_target_lists)
total_target_lists = np.reshape(total_target_lists , (-1,1))
print(total_data.shape)
total_data = np.concatenate((total_data, total_target_lists), axis=1)
print(np.random.shuffle(total_data))
print(total_data.shape)


print("done")

6381.437304203095
4251.753897920579
6144.11415741369
4115.876626022915
8022.71122980362
6982.567977574119
6715.78205367344
7974.525465078601
6218.165712362113
6637.455572241806
9735.200951825951
4678.119723764315
8461.325668377365
4711.451699595217
5056.789766357645
9661.526398780394
10996.767941073536
15615.565847744443
13185.132387325188
13304.320337808447
12385.869870292532
10811.71937481932
11047.548758199928
11167.439464964002
(24000, 199)
None
(24000, 200)
done


In [103]:
# Import train_test_split function
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(total_data[:,0:199], total_data[:,199], train_size=0.65,test_size=0.35, random_state=101) # 70% training and 30% test
# print (X_train)
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
print (X_test.shape)
y_pred = clf.predict(X_test)
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

(8400, 199)
Accuracy: 0.964047619047619


In [121]:
t , h, e , h_n , e_n = plot_hist("test_waleed3_17_7",0)
# print(t,e,h.size)
# f = [ h[n] if e[n] < t else 0 for n in range(h.size)]
# print(f)
# f =  100 *(f / np.sum(f))
# print(f)
f = (h_n - np.min(h_n)) /(np.max(h_n) - np.min(h_n))
# f = np.trim_zeros(f)
# # print(f)
# N = 59 - len(f)
# f = np.pad(f, (0, N), 'constant')
# print(f)
data_list = np.array([f])
y_pred = clf.predict(data_list)
# y_pred[1600]
print(y_pred)

[1.]


In [105]:
result_p = []
result_n = []
count = 0
for i in range(1000):
    t , h, e , h_n , e_n = plot_hist("noisy_air_"+str(i),1)
    # print(t,e,h.size)
    # f = [ h[n] if e[n] < t else 0 for n in range(h.size)]
    # print(f)

    # f =  100 *(f / np.sum(f))
    # print(f)
    f = (h_n - np.min(h_n)) /(np.max(h_n) - np.min(h_n))
    # f = np.trim_zeros(f)
    # # print(f)
    # N = 59 - len(f)
    # f = np.pad(f, (0, N), 'constant')
    # print(f)
    data_list = np.array([f])
    y_pred = clf.predict(data_list)
    count +=1
    print("count :: ",count)
    if y_pred == 1:
        result_p.append(y_pred)
    else:
        result_n.append(y_pred)
        
        
print("number of positive results :: ", len(result_p))
print("number of negative results :: ", len(result_n))
print("positive accuracy :: ", (100 *len(result_p)/(len(result_p)+len(result_n)) ))
print("negative accuracy :: ", (100 *len(result_n)/(len(result_p)+len(result_n)) ))

count ::  1
count ::  2
count ::  3
count ::  4
count ::  5
count ::  6
count ::  7
count ::  8
count ::  9
count ::  10
count ::  11
count ::  12
count ::  13
count ::  14
count ::  15
count ::  16
count ::  17
count ::  18
count ::  19
count ::  20
count ::  21
count ::  22
count ::  23
count ::  24
count ::  25
count ::  26
count ::  27
count ::  28
count ::  29
count ::  30
count ::  31
count ::  32
count ::  33
count ::  34
count ::  35
count ::  36
count ::  37
count ::  38
count ::  39
count ::  40
count ::  41
count ::  42
count ::  43
count ::  44
count ::  45
count ::  46
count ::  47
count ::  48
count ::  49
count ::  50
count ::  51
count ::  52
count ::  53
count ::  54
count ::  55
count ::  56
count ::  57
count ::  58
count ::  59
count ::  60
count ::  61
count ::  62
count ::  63
count ::  64
count ::  65
count ::  66
count ::  67
count ::  68
count ::  69
count ::  70
count ::  71
count ::  72
count ::  73
count ::  74
count ::  75
count ::  76
count ::  77
count ::

count ::  603
count ::  604
count ::  605
count ::  606
count ::  607
count ::  608
count ::  609
count ::  610
count ::  611
count ::  612
count ::  613
count ::  614
count ::  615
count ::  616
count ::  617
count ::  618
count ::  619
count ::  620
count ::  621
count ::  622
count ::  623
count ::  624
count ::  625
count ::  626
count ::  627
count ::  628
count ::  629
count ::  630
count ::  631
count ::  632
count ::  633
count ::  634
count ::  635
count ::  636
count ::  637
count ::  638
count ::  639
count ::  640
count ::  641
count ::  642
count ::  643
count ::  644
count ::  645
count ::  646
count ::  647
count ::  648
count ::  649
count ::  650
count ::  651
count ::  652
count ::  653
count ::  654
count ::  655
count ::  656
count ::  657
count ::  658
count ::  659
count ::  660
count ::  661
count ::  662
count ::  663
count ::  664
count ::  665
count ::  666
count ::  667
count ::  668
count ::  669
count ::  670
count ::  671
count ::  672
count ::  673
count 

In [ ]:

vas_data_list , vas_target_list = pre_processing("vas")

print (vas_target_list.shape)
print (vas_data_list.shape)

print (type(vas_data_list[1]))
print ([vas_data_list[1].tolist()])

In [ ]:
# print (X_test[1600])
y_pred = clf.predict(np.array([vas_data_list[0].tolist()]))
# y_pred[1600]
print(y_pred)

In [ ]:
import numpy as np
import open3d as o3d
folder_indx = 0 
exp_name = "waleed_14_7_136"
folder = ["3D_Experements", "flat_Experements"]
dist = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_x.txt")
upper_angle = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_y.txt")
lower_angle = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_z.txt")



#x , y , z = np.array(dist)*np.cos(uAngel)*np.sin(lAngel) , np.array(dist)*np.cos(uAngel)*np.cos(lAngel) , np.array(dist)*np.sin(uAngel)

my_sample_x = np.array(dist)*np.cos(upper_angle)*np.sin(lower_angle)
my_sample_y =  np.array(dist)*np.cos(upper_angle)*np.cos(lower_angle)
my_sample_z =  np.array(dist)*np.sin(upper_angle)
my_sample_x = my_sample_x[~np.isnan(my_sample_x)]
my_sample_y = my_sample_y[~np.isnan(my_sample_y)]
my_sample_z = my_sample_z[~np.isnan(my_sample_z)]

# print (type(my_sample_z.tolist()))

points = np.array([my_sample_x.tolist(), my_sample_y.tolist(), my_sample_z.tolist()])
print(points.shape)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[:,:3])
# o3d.visualization.draw_geometries([pcd])
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))
# o3d.visualization.draw_geometries([bpa_mesh])
o3d.visualization.draw_geometries(bpa_mesh)
# dec_mesh = o3d.geometry.simplify_quadric_decimation(100000)
# dec_mesh.remove_degenerate_triangles()
# dec_mesh.remove_duplicated_triangles()
# dec_mesh.remove_duplicated_vertices()
# dec_mesh.remove_non_manifold_edges()

In [ ]:
import plotly.graph_objects as go
import numpy as np
  
# Data for three-dimensional scattered points
# z = 15 * np.random.random(100)
# x = np.sin(z) + 0.1 * np.random.randn(100)
# y = np.cos(z) + 0.1 * np.random.randn(100)
folder_indx = 0
exp_name = "ta7a_14_7_136"
folder = ["3D_Experements", "flat_Experements"]
dist = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_x.txt")
upper_angle = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_y.txt")
lower_angle = np.loadtxt(folder[folder_indx]+"/"+exp_name+"_z.txt")



#x , y , z = np.array(dist)*np.cos(uAngel)*np.sin(lAngel) , np.array(dist)*np.cos(uAngel)*np.cos(lAngel) , np.array(dist)*np.sin(uAngel)

my_sample_x = np.array(dist)*np.cos(upper_angle)*np.sin(lower_angle)
my_sample_y =  np.array(dist)*np.cos(upper_angle)*np.cos(lower_angle)
my_sample_z =  np.array(dist)*np.sin(upper_angle)
my_sample_x = my_sample_x[~np.isnan(my_sample_x)]
my_sample_y = my_sample_y[~np.isnan(my_sample_y)]
my_sample_z = my_sample_z[~np.isnan(my_sample_z)]
fig = go.Figure(data=[go.Mesh3d(x=my_sample_x, y=my_sample_y, z=my_sample_z, color='green',
                                opacity=0.80, alphahull=3)])
  
fig.show()